# Keras colorizer of CelebA using Generative Adversarial Networks.
The dataset can be downloaded from: https://www.dropbox.com/sh/8oqt9vytwxb3s4r/AADIKlz8PR9zr6Y20qbkunrba/Img/img_align_celeba.zip?dl=0
## Instrustion on running the script:
1. Download the dataset from the provided link
2. Save the folder 'img_align_celeba' to 'datasets/'

In [3]:
import tensorflow as tf

In [8]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! mkdir datasets
! mkdir originals

In [ ]:
! unzip -q "/content/drive/My Drive/img_align_celeba.zip" -d datasets

In [ ]:
! unzip -q "/content/drive/My Drive/coco_val2017.zip" -d datasets

In [ ]:
! mv datasets/val2017/* originals

In [ ]:
! find /content/datasets/img_align_celeba -type f -exec sh -c 'mv "$@" "$0"' originals/ {} +

In [1]:
import scipy

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, MaxPooling2D
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
import keras
#from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from glob import glob
import keras.backend as K
import scipy.misc
import PIL
import os
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.optimizers import Adam

Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [31]:
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 3} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [32]:
class DataLoader():
  def __init__(self, img_res=(256, 256)):
    self.img_res = img_res

  def load_data(self, batch_size=1, is_testing=False):
    path = glob('./originals/*')
    batch_images = np.random.choice(path, size=batch_size)

    imgs_hr = []
    imgs_lr = []
    for img_path in batch_images:
      img_hr, img_lr = self._load(img_path, self.img_res)

      imgs_hr.append(img_hr)
      imgs_lr.append(img_lr)
      
    # нормализация данных
    imgs_hr = np.array(imgs_hr) / 127.5 - 1
    imgs_lr = np.array(imgs_lr) / 127.5 - 1

    return imgs_hr, imgs_lr

  # returns pair (original photo, grayscale photo)
  def _load(self, path, size):
    return np.array(PIL.Image.open(path).resize(size)).astype(np.float), np.expand_dims(np.array(PIL.Image.open(path).resize(size).convert('L')).astype(np.float), axis=2)

In [33]:
hr_channels = 3
lr_channels = 1
width = 512
lr_shape = (width, width, lr_channels)
hr_shape = (width, width, hr_channels)

n_residual_blocks = 16
optimizer = Adam(0.0002, 0.5)

# configure data loader
data_loader = DataLoader(img_res=(width, width))

In [34]:
def sample_images(epoch):
  os.makedirs('samples', exist_ok=True)
  r,c = 2,3
  imgs_hr, imgs_lr = data_loader.load_data(batch_size=2, is_testing=True)
  fake_hr = generator.predict(imgs_lr)
  imgs_lr = 0.5 * imgs_lr + 0.5
  fake_hr = 0.5 * fake_hr + 0.5
  imgs_hr = 0.5 * imgs_hr + 0.5
  titles = ["B&W", "Generated", "Original"]
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for row in range(r):
    for col, image in enumerate([imgs_lr, fake_hr, imgs_hr]):
      if col == 0:
        axs[row, col].imshow(np.array(PIL.Image.fromarray((np.squeeze(image[row])*255).astype(np.uint8)).convert('RGB')))
      else:
        axs[row, col].imshow(image[row])
      axs[row, col].set_title(titles[col])
      axs[row, col].axis('off')
    cnt += 1
  def zerofy(s: str):
    while len(s) < 6:
      s = '0' + s
    return s
  fig.savefig('samples/'+zerofy(str(epoch)) +'.png')
  plt.close()

We use a pre-trained VGG19 model to extract image features from the high resolution and the generated high resolution images and minimize the mse between them

In [35]:
def build_feature_loss(input_size):
  vgg = VGG19(weights='imagenet')
  vgg.outputs = [vgg.layers[9].output]
  img = Input(shape=input_size)
  img_features = vgg(img)
  model = Model(img, img_features)
  model.trainable = False 
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
  return model
feature_loss = build_feature_loss(hr_shape)

build and compile the discriminator

In [36]:
def build_discriminator(input_size):
  # Number of filters in the first layer of G and D
  gf = 64
  df = 64
  def d_block(layer_input, filters, strides=1, bn=True):
    """Discriminator layer"""
    d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if bn:
        d = BatchNormalization(momentum=0.8)(d)
    return d

  # Input img
  d0 = Input(shape=input_size)

  d1 = d_block(d0, df, bn=False)
  d2 = d_block(d1, df, strides=2)
  d3 = d_block(d2, df*2)
  d4 = d_block(d3, df*2, strides=2)
  d5 = d_block(d4, df*4)
  d6 = d_block(d5, df*4, strides=2)
  d7 = d_block(d6, df*8)
  d8 = d_block(d7, df*8, strides=2)

  d9 = Dense(df*16)(d8)
  d10 = LeakyReLU(alpha=0.2)(d9)
  validity = Dense(1, activation='sigmoid')(d10)

  model = Model(d0, validity)
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
  return model

discriminator = build_discriminator(input_size=hr_shape)

Build the generator

In [37]:
def unet(pretrained_weights = None, input_size = (256, 256, 3)):
  inputs = Input(input_size)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(3, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(3, 1, activation = 'tanh')(conv9)

  model = Model(input = inputs, output = conv10)
  
  #model.summary()

  if(pretrained_weights):
    model.load_weights(pretrained_weights)
  model.compile(optimizer = Adam(lr = 1e-4), loss = 'mse', metrics = ['accuracy'])
  return model
generator = unet(input_size=lr_shape)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [38]:
def build_gan(input_size):
  # High res. and low res. images
  img_lr = Input(shape=lr_shape)
  # generate high res. version from low res.
  fake_hr = generator(img_lr)
  # extract image features of the generated img
  fake_features = feature_loss(fake_hr)
  # for the combined model we will only train the generator
  discriminator.trainable = False
  # Discriminator determines validity of generated high res. images
  validity = discriminator(fake_hr)
  combined = Model([img_lr], [validity, fake_features])
  combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1e-3, 1], optimizer=optimizer)
  return combined
gan = build_gan(lr_shape)

## Train

In [62]:
#! rm -rf ./samples
epochs=30000
batch_size=1
sample_interval=50
start_time = datetime.datetime.now()
for epoch in range(epochs+1):
  # calculate output shape of D (PatchGAN)
  patch = int(width / 2**4)
  disc_patch = (patch, patch, 1)
  # ----------------------
  #  Train Discriminator
  # ----------------------

  # Sample images and their conditioning counterparts
  imgs_hr, imgs_lr = data_loader.load_data(batch_size)

  # From low res. image generate high res. version
  fake_hr = generator.predict(imgs_lr)

  valid = np.ones((batch_size,) + disc_patch)
  fake = np.zeros((batch_size,) + disc_patch)

  # Train the discriminators (original images = real / generated = Fake)
  d_loss_real = discriminator.train_on_batch(imgs_hr, valid)
  d_loss_fake = discriminator.train_on_batch(fake_hr, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ------------------
  #  Train Generator
  # ------------------

  # Sample images and their conditioning counterparts
  imgs_hr, imgs_lr = data_loader.load_data(batch_size)

  # The generators want the discriminators to label the generated images as real
  valid = np.ones((batch_size,) + disc_patch)

  # Extract ground truth image features using pre-trained VGG19 model
  image_features = feature_loss.predict(imgs_hr)

  # Train the generators
  g_loss = gan.train_on_batch([imgs_lr], [valid, image_features])

  elapsed_time = datetime.datetime.now() - start_time
  # Plot the progress
  print ("%d time: %s" % (epoch, elapsed_time))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      sample_images(epoch)

0 time: 0:00:01.630639
1 time: 0:00:04.039198
2 time: 0:00:05.595058
3 time: 0:00:07.168828
4 time: 0:00:08.724667
5 time: 0:00:10.254598
6 time: 0:00:11.792461
7 time: 0:00:13.338349
8 time: 0:00:14.862251
9 time: 0:00:16.386204
10 time: 0:00:17.919142
11 time: 0:00:19.434062
12 time: 0:00:20.962977
13 time: 0:00:22.493879
14 time: 0:00:24.006833
15 time: 0:00:25.518816
16 time: 0:00:27.046702
17 time: 0:00:28.567656
18 time: 0:00:30.083580
19 time: 0:00:31.604512
20 time: 0:00:33.134420
21 time: 0:00:34.665325
22 time: 0:00:36.189277
23 time: 0:00:37.715209
24 time: 0:00:39.249084
25 time: 0:00:40.767024
26 time: 0:00:42.285962
27 time: 0:00:43.800932
28 time: 0:00:45.329821
29 time: 0:00:46.866737
30 time: 0:00:48.394646
31 time: 0:00:49.929519
32 time: 0:00:51.454442
33 time: 0:00:52.978369
34 time: 0:00:54.498300
35 time: 0:00:56.025216
36 time: 0:00:57.550163
37 time: 0:00:59.073065
38 time: 0:01:00.603970
39 time: 0:01:02.137889
40 time: 0:01:03.660795
41 time: 0:01:05.187739
42

ValueError: Error when checking input: expected input_18 to have 4 dimensions, but got array with shape (1, 512, 512)

In [30]:
generator

In [63]:
save_generator = keras.Sequential([generator])

In [64]:
save_generator.add(Lambda(lambda x: (x + 1) * 127.5))

## Convert to CoreML

In [65]:
save_generator.save('generator.h5')

In [12]:
! pip install coremltools==3.2

     |████████████████████████████████| 4.0MB 686kB/s eta 0:00:01
  Found existing installation: coremltools 3.4
    Uninstalling coremltools-3.4:
      Successfully uninstalled coremltools-3.4


In [5]:
! pip install keras==2.2.4
! pip install tensorflow==1.14.0

     |████████████████████████████████| 317kB 158kB/s eta 0:00:01
     |████████████████████████████████| 105.8MB 263kB/s eta 0:00:01
     |████████████████████████████████| 3.2MB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 491kB 2.7MB/s eta 0:00:01
  Found existing installation: tensorboard 2.0.0
    Uninstalling tensorboard-2.0.0:
      Successfully uninstalled tensorboard-2.0.0
  Found existing installation: tensorflow-estimator 2.0.0
    Uninstalling tensorflow-estimator-2.0.0:
      Successfully uninstalled tensorflow-estimator-2.0.0
  Found existing installation: tensorflow 2.0.0
    Uninstalling tensorflow-2.0.0:
      Successfully uninstalled tensorflow-2.0.0


In [1]:
from coremltools.proto import NeuralNetwork_pb2

def convert_lambda(layer):
    params = NeuralNetwork_pb2.CustomLayerParams()

    # The name of the Swift or Obj-C class that implements this layer.
    params.className = "Lambda"

    # The desciption is shown in Xcode's mlmodel viewer.
    params.description = "Post process"

    return params

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

In [15]:
import keras

Using TensorFlow backend.


In [ ]:
! python

Python 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)] :: Anaconda, Inc. on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [2]:
import coremltools
coreml_model = coremltools.converters.keras.convert('generator.h5', input_names='input', output_names='output', image_input_names='input', add_custom_layers=True, image_scale=1/127.5, gray_bias=-1, custom_conversion_functions={ "Lambda": convert_lambda })

# Saving the Core ML model to a file.
coreml_model.save('colorizer.mlmodel')


/opt/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


0 : model_15_input, <keras.engine.input_layer.InputLayer object at 0x1a33d54590>
1 : model_15_conv2d_105, <keras.layers.convolutional.Conv2D object at 0x1a33d54a50>
2 : model_15_conv2d_105__activation__, <keras.layers.core.Activation object at 0x1a343b81d0>
3 : model_15_conv2d_106, <keras.layers.convolutional.Conv2D object at 0x1a33d54b10>
4 : model_15_conv2d_106__activation__, <keras.layers.core.Activation object at 0x1a33d3e050>
5 : model_15_max_pooling2d_13, <keras.layers.pooling.MaxPooling2D object at 0x1a33d54d90>
6 : model_15_conv2d_107, <keras.layers.convolutional.Conv2D object at 0x1a33d54bd0>
7 : model_15_conv2d_107__activation__, <keras.layers.core.Activation object at 0x1a33d3e350>
8 : model_15_conv2d_108, <keras.layers.convolutional.Conv2D object at 0x1a33d8c150>
9 : model_15_conv2d_108__activation__, <keras.layers.core.Activation object at 0x1a33d3e290>
10 : model_15_max_pooling2d_14, <keras.layers.pooling.MaxPooling2D object at 0x1a33d8c350>
11 : model_15_conv2d_109, <ker

/opt/anaconda3/lib/python3.7/site-packages/coremltools/models/model.py:111: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "compiler error:  Error creating Core ML custom layer implementation from factory for layer "Lambda".".
  RuntimeWarning)


In [3]:
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft 

spec = coremltools.utils.load_spec("colorizer.mlmodel")

In [5]:
output = spec.description.output[0]
width = 512
import coremltools.proto.FeatureTypes_pb2 as ft
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB
output.type.imageType.height = width
output.type.imageType.width = width

coremltools.utils.save_spec(spec, "Colorizer2.mlmodel")

In [ ]:
! pip uninstall keras
! pip uninstall tensorflow
! pip install tensorflow
! pip install keras

Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.4
Uninstalling tensorflow-1.14.0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.14.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.14.0
     |████████████████████████████████| 516.2MB 34kB/s 
     |████████████████████████████████| 3.0

     |████████████████████████████████| 378kB 2.8MB/s eta 0:00:01
